In [10]:
from collections import deque
from dataclasses import dataclass, field
from itertools import islice
from typing import Dict, Callable, Deque, Optional

from pyprojroot import here

In [83]:
@dataclass
class Monkey:
    items: Deque[int]
    inspect: Callable[[int], int]
    reduceWorry: Callable[[int], int]
    throwPredicate: Callable[[int], bool]
    trueMonkey: int
    falseMonkey: int
    monkeys: 'Monkeys'
    itemsInspected = 0

    def throw(self, item: int):
        item = self.inspect(item)
        item = self.reduceWorry(item)
        self.itemsInspected += 1
        
        if self.throwPredicate(item):
            self.monkeys.monkeys[self.trueMonkey].receive(item)
        else:
            self.monkeys.monkeys[self.falseMonkey].receive(item)

    def receive(self, item: int):
        self.items.append(item)

    def takeTurn(self):
        while self.items:
            self.throw(self.items.popleft())


@dataclass
class Monkeys:
    monkeys: Dict[int, 'Monkey'] = field(default_factory=dict)

    def add(self, monkey: Dict[int, Monkey]):
        self.monkeys.update(monkey)

    def round(self):
        for monkey in self.monkeys.values():
            monkey.takeTurn()

In [93]:
path = here('./11/input-1.txt')
with open(path, 'r') as fp:
    monkeysDict = {}
    monkeyNum = -1
    monkeys = Monkeys()

    while True:
        monkey = [line.strip() for line in islice(fp, 7)]

        if monkey:
            # items
            items = deque([int(char) for char in monkey[1].split(': ')[1].split(', ')])

            # inspect
            if '*' in monkey[2]:
                argStr = monkey[2].split(' * ')[1]
                if argStr == 'old':
                    inspect = lambda x: x * x
                else:
                    inspect = (lambda arg: lambda x: x * arg)(int(argStr))
            else:
                arg = int(monkey[2].split(' + ')[1])
                inspect = (lambda arg: lambda x: x + arg)(arg)

            # throw predicate
            arg = int(monkey[3].split('by ')[1])
            throwPredicate = (lambda arg: lambda x: x % arg == 0)(arg)

            # true monkey
            trueMonkey = int(monkey[4].split('monkey ')[1])

            # false monkey
            falseMonkey = int(monkey[5].split('monkey ')[1])
            
            monkeyNum += 1
            monkeys.add({
                monkeyNum: Monkey(
                    items,
                    inspect,
                    lambda x: x // 3,
                    throwPredicate,
                    trueMonkey,
                    falseMonkey,
                    monkeys
                )
            })
        else:
            break

In [94]:
for _ in range(20):
    monkeys.round()

In [95]:
[monkey.itemsInspected for monkey in monkeys.monkeys.values()]

[327, 48, 320, 61, 11, 345, 296, 18]

In [88]:
345 * 327

112815